In [ ]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [ ]:
import os
import shutil
import random
from tqdm.notebook import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
import numpy as np
import cv2
from ultralytics import YOLO
import os

# paths to directories and model weights
image_dir = '/content/gdrive/MyDrive/object_detection/datasets/test2/images'
output_dir = '/content/gdrive/MyDrive/object_detection/datasets2/inference2'
person_weights = '/content/gdrive/MyDrive/object_detection/datasets2/training_results/person_detection_22/weights/best.pt'
ppe_weights = '/content/gdrive/MyDrive/object_detection/datasets2/training_results/ppe_detection_22/weights/best.pt'

# function to generate a color for each class
def get_color_for_class(class_id):
    np.random.seed(class_id)  # Seed for reproducibility
    return tuple(np.random.randint(0, 255, 3).tolist())

# function for inference
def run_inference(image_dir, person_weights, ppe_weights, output_dir):
    # load both the models
    person_model = YOLO(person_weights)
    ppe_model = YOLO(ppe_weights)

    os.makedirs(output_dir, exist_ok=True) # if output folder does not exist, create it

    # iterate over all images in the directory
    for image_file in os.listdir(image_dir):
        if not image_file.endswith(('.jpg', '.jpeg', '.png')):
            continue

        image_path = os.path.join(image_dir, image_file)
        image = cv2.imread(image_path)

        # inference for person detection
        results = person_model(image)

        for result in results:  # loop over each detection result
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # extract the bounding box coordinates
                person_crop = image[y1:y2, x1:x2]  # crop the person region for PPE detection

                # inference for PPE detection on the cropped image
                ppe_results = ppe_model(person_crop)

                for ppe_result in ppe_results:  # loop over each PPE detection result
                    for ppe_box in ppe_result.boxes:
                        px1, py1, px2, py2 = map(int, ppe_box.xyxy[0])  # extract the bounding box coordinates for PPE

                        # adjust the PPE box coordinates to match the original image
                        px1 += x1
                        py1 += y1
                        px2 += x1
                        py2 += y1

                        color = get_color_for_class(int(ppe_box.cls))  # get color for PPE class

                        # draw bounding boxes on the original image
                        label = ppe_model.names[int(ppe_box.cls)]
                        conf = float(ppe_box.conf)  # convert tensor to float
                        cv2.rectangle(image, (px1, py1), (px2, py2), color, 2)
                        cv2.putText(image, f'{label} {conf:.2f}', (px1, py1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # draw bounding boxes for detected persons
        for result in results:  # loop over each detection result
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # Extract the bounding box coordinates
                color = (255, 0, 0)  # Red color for person detection

                # draw bounding boxes on the original image
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
                cv2.putText(image, 'Person', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # save the final image with bounding boxes
        output_path = os.path.join(output_dir, image_file)
        cv2.imwrite(output_path, image)

# run the inference
run_inference(image_dir, person_weights, ppe_weights, output_dir)


0: 448x640 3 persons, 1069.7ms
Speed: 4.6ms preprocess, 1069.7ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)

0: 640x192 1 hard-hat, 2 bootss, 484.1ms
Speed: 1.7ms preprocess, 484.1ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 192)

0: 640x288 2 hard-hats, 2 bootss, 709.1ms
Speed: 2.7ms preprocess, 709.1ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 288)

0: 640x288 2 bootss, 708.9ms
Speed: 3.1ms preprocess, 708.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 288)

0: 448x640 3 persons, 1248.8ms
Speed: 4.3ms preprocess, 1248.8ms inference, 1.2ms postprocess per image at shape (1, 3, 448, 640)

0: 544x640 2 hard-hats, 1 gloves, 2 bootss, 2 vests, 2045.5ms
Speed: 3.7ms preprocess, 2045.5ms inference, 1.3ms postprocess per image at shape (1, 3, 544, 640)

0: 640x288 1 vest, 1142.0ms
Speed: 2.4ms preprocess, 1142.0ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 288)

0: 640x352 1 boots, 1 vest, 1218.4ms
S